In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
from problems import *

In [ ]:
import os
from functools import partial
from torch.utils.data import DataLoader

In [ ]:
from scripts.create_problem import load_problem_set
import problems as pbs
from dpt.data import *
from dpt.reward import Reward
from train_dpt import *
from utils import *

In [ ]:
problem = 'Normal(0, 1)'
data_dir = '../data/normal_10'
results_dir = '../results/normal_10'
suffix = 'test'

problems = load_problem_set(os.path.join(data_dir, problem, suffix))
dataset = OfflineDataset(problems, seq_len=100, results_dir=results_dir, suffix=suffix, ad_ratio=0.0, action='bitflip', target_action='gt')
# dataset = OnlineDataset(problems)
sample = dataset[0]

for k, v in sample.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
import matplotlib.pyplot as plt
target_actions = [torch.argmax(sample['target_action']).item() for _ in range(10) for sample in dataset]
bins = np.arange(0, 12)
plt.hist(target_actions, bins-0.5)
plt.xticks(bins[:-1])
plt.show()

In [ ]:
reward_model = Reward()
rewards = reward_model.offline(sample['states'], sample['actions'], sample['next_states'])

In [ ]:
torch.all(sample['states'][..., :-1].long() ^ sample['actions'][..., :-1] == sample['next_states'][..., :-1].long())

In [ ]:
sample = dataset[0]
print_sample(sample, rewards, 'bitflip')

In [ ]:
collate_fn = partial(custom_collate_fn, problem_class=getattr(pbs, 'Problem'))
dataloader = DataLoader(dataset=dataset, batch_size=10, collate_fn=collate_fn)
batch = next(iter(dataloader))

for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
# from train_dpt import ProblemDataModule, DPTSolver, L
from utils import load_config

In [ ]:
config = load_config('../configs/config.yaml')
# config['batch_size'] = 3

# config['action'] = "point"
# # target_action: "gt"
# # ad_ratio: 0.0
# config['model_params']['action_dim'] = 50

datamodule = ProblemDataModule(config)
datamodule.setup()

In [ ]:
batch = next(iter(datamodule.test_dataloader()))

In [ ]:
model = DPTSolver(config)
model.configure_model()

In [ ]:
model.test_step(batch, 0)

In [ ]:
# rewards = model.reward_model.offline(
rewards = reward_model.offline(
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"]
)
rewards.shape

In [ ]:
outputs = model.model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=rewards,
)
outputs.shape

In [ ]:
trainer = L.Trainer(
    logger=None,
    precision=config["precision"] if config["strategy"] != "fsdp" else None,
    max_epochs=config["max_epochs"],
    log_every_n_steps=config["log_every_n_steps"],
    default_root_dir=config["wandb_params"]["save_dir"],
    enable_model_summary=True,
    use_distributed_sampler=False,
    # strategy=config["strategy"]
)